In [ ]:
from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
import numpy as np # Pythons standard array library
import xarray as xr # Libary to work with labeled n-dimensional data
import sys
import os
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd

# 1st step: Load original data

In [ ]:
#Define files in my work place
glob_pattern_land = 'bc_land_0505'
## Define the paths of file 1
data_path = Path('/work/mh0731/m300948/AMDEF/REGRID_BC')
file1 = sorted([str(f) for f in data_path.glob(f'*{glob_pattern_land}*.nc')])[:]

In [ ]:
#Define files in my work place
glob_pattern_land = 'bc_land_soil_regrid'
## Define the paths of file 1
data_path = Path('/work/mh0731/m300948/AMDEF/REGRID_BC')
file1 = sorted([str(f) for f in data_path.glob(f'*{glob_pattern_land}*.nc')])[:]

In [ ]:
#Define files in my work place
glob_pattern_land = 'bc_land_frac_regrid'
## Define the paths of file 1
data_path = Path('/work/mh0731/m300948/AMDEF/REGRID_BC/amz_100def/')
file1 = sorted([str(f) for f in data_path.glob(f'*{glob_pattern_land}*.nc')])[:]

# 2nd Step: Load biome masking data

In [ ]:
file_m1 = '/work/mh0731/m300948/AMDEF/amz_100def/biome_100def.nc'
dset_m1 = xr.open_dataset(file_m1) # if it is ncfile
var_names = ['lai_clim','veg_fract','roughness_length','albedo']
dset_m1_subset = dset_m1[var_names].persist()
dset_m1_subset

In [ ]:
file_m1 = '/work/mh0731/m300948/AMDEF/masking_files/biome_root_100def.nc'
dset_m1 = xr.open_dataset(file_m1) # if it is ncfile
var_names = ['root_depth']
dset_m1_subset = dset_m1[var_names].persist()
dset_m1_subset

In [ ]:
file_m1 = '/work/mh0731/m300948/AMDEF/masking_files/biome_vegf_100def.nc'
dset_m1 = xr.open_dataset(file_m1) # if it is ncfile
var_names = ['veg_ratio_max']
dset_m1_subset = dset_m1[var_names].persist()
dset_m1_subset

# 3rd Step: Interpolation

In [ ]:
####### Basin/Biome masking file from GIS
#ABinterp_biome = dset_mask_biome.interp(latitude=dset_subset.lat, longitude=dset_subset.lon)
#AB_biome=ABinterp_biome['AMAZON_BIOMES'].to_numpy()

####### Biome masking file from original file - interpolate with LAI data
ABinterp_lai = dset_mask_lai.interp(lat=dset_m1_subset.lat, lon=dset_m1_subset.lon)
AB_lai=dset_mask_lai['lai_clim'].to_numpy()